In [1]:
# Import the libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import keras
from keras.models import Sequential
from keras.layers import Dense

import matplotlib.pyplot as plt

In [2]:
# Download data and split training & test sets 
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
# Split columns into Predictors and Target
concrete_data_columns = concrete_data.columns

P = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
T = concrete_data['Strength'] # Strength column

In [4]:
# Mormalise the Predictors
P_norm = (P - P.mean()) / P.std()
P_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [5]:
# number of predictors
n_cols = P_norm.shape[1] 

In [6]:
# Define the regression model with one hidden layer
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Build the model
model = regression_model()

In [7]:
# We create a 50-cycle training & evaluation loop. We would train the model on a 100-epoch fit -per cycle- and evaluate using MSE per cycle. We record each cycle MSE into a list.

score_mse_list = [] # Create a list to record the MSEs per cycle  
cycle  = 0 # Initialise the cycles

for looping in range(50): # Loop 50 times
    
    # Split the Predictors & Target data, holding 30% for testing
    P_norm_train, P_norm_test, T_train, T_test = train_test_split(P_norm, T,
                                                    test_size = 0.3,
                                                    random_state = 1)
    #Initialise the cycle MSE
    score_mse = 0
    
    # Fit the model 
    model.fit( P_norm_train, T_train, epochs=100, verbose=2)
    
    # Generate predicted test target values
    T_test_hat = model.predict(P_norm_test)
    
    # Cycle evaluation using scikit-learn mean_squared_error
    score_mse = mean_squared_error(T_test, T_test_hat) 
    
    # Append the loop MSE on the list
    score_mse_list.append(score_mse)

Epoch 1/100
23/23 - 1s - loss: 1521.4283 - 622ms/epoch - 27ms/step
Epoch 2/100
23/23 - 0s - loss: 1502.0590 - 22ms/epoch - 957us/step
Epoch 3/100
23/23 - 0s - loss: 1482.8425 - 23ms/epoch - 1000us/step
Epoch 4/100
23/23 - 0s - loss: 1462.7208 - 21ms/epoch - 913us/step
Epoch 5/100
23/23 - 0s - loss: 1442.1631 - 21ms/epoch - 917us/step
Epoch 6/100
23/23 - 0s - loss: 1420.4204 - 21ms/epoch - 913us/step
Epoch 7/100
23/23 - 0s - loss: 1398.2845 - 23ms/epoch - 1ms/step
Epoch 8/100
23/23 - 0s - loss: 1375.0265 - 30ms/epoch - 1ms/step
Epoch 9/100
23/23 - 0s - loss: 1351.1172 - 21ms/epoch - 921us/step
Epoch 10/100
23/23 - 0s - loss: 1326.0636 - 21ms/epoch - 913us/step
Epoch 11/100
23/23 - 0s - loss: 1299.7819 - 20ms/epoch - 873us/step
Epoch 12/100
23/23 - 0s - loss: 1272.6049 - 19ms/epoch - 830us/step
Epoch 13/100
23/23 - 0s - loss: 1244.6019 - 19ms/epoch - 826us/step
Epoch 14/100
23/23 - 0s - loss: 1215.2483 - 20ms/epoch - 870us/step
Epoch 15/100
23/23 - 0s - loss: 1185.7135 - 20ms/epoch - 870

In [8]:
# Printing the list of MSEs and verify the its length
print(score_mse_list)
len(score_mse_list)

[177.97557830617893, 99.20275166808733, 76.37646195063547, 65.18306829463613, 59.652697622028384, 57.37346741148723, 55.32661355508657, 53.53826236419797, 52.85784932906101, 51.95328987024115, 50.932236478072724, 50.11951133010813, 49.4570582266012, 49.27891685164631, 48.918928208046076, 48.50515909460878, 48.58557641876756, 48.26316341413862, 48.21573736749432, 47.87113512414533, 47.53847463324854, 47.60284498805159, 47.57787935248339, 47.38987817305967, 47.27045504678959, 47.121347370515664, 47.11226428963447, 46.9205816084656, 46.94026552749108, 46.98722294400459, 46.94755782188039, 46.8947001749508, 46.7600299237043, 46.47733470386279, 46.44966883131197, 46.19148949179075, 46.125513015239775, 46.29002831457052, 46.255724693503836, 46.14243053466075, 46.263568978374344, 46.148017226053575, 46.05437922964433, 45.825339631961725, 45.93521281969465, 45.96257179663213, 45.76199726781376, 45.5435953589108, 44.480760702802115, 43.780645945136385]


50

In [9]:
# Calculate and report the mean & std of the MSEs from the list
print('The training values mean of the 50-cycle MSEs is :', np.mean(score_mse_list))
print('The training values standard deviation of the 50-cycle MSEs is :', np.std(score_mse_list))

The training values mean of the 50-cycle MSEs is : 52.64678486563025
The training values standard deviation of the 50-cycle MSEs is : 19.998798006187062


## The mean and the standard deviation of the MSEs have decreased significantly after increasing to 100 epochs per cycle.